In [23]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [2]:
# .env 파일 로드
load_dotenv()

# 환경 변수에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")


In [6]:
# 파일 로드
file_path = "비상시국민행동요령.PDF"
# PDF 파일 경로
loader = PyPDFLoader(file_path=file_path)

docs = loader.load()

In [7]:
page_number = docs[1].metadata['page']
print(page_number)
page_text = docs[0].page_content
print(page_text)
print(list(docs[0].metadata.keys())) 

1
비상시 국민행동요령
CONTENTS
온 가족이 함께 
안전하게
비상사태시 행동요령
화생방 피해대비 
행동요령
인명·시설 피해시 
행동요령
비상대비물자 
준비 및 사용요령
•만화(웹툰)로 보는  02
    비상시 국민행동요령
•비상시 행동요령 08
•민방공 경보 발령시 행동요령 11
•일상생활 비상대비 3가지 14
•화학무기 피해대비 행동요령 20
•생물학무기 피해대비 행동요령 21 
•핵·방사능 피폭대비 행동요령 23
•핵·방사능 피폭대비 생존상식 25
•대형건물 붕괴·화재시 행동요령 32
•전기·물·가스 공급 중단시 행동요령 34
•지하철 피해시 행동요령 35
•인명·시설 피해복구 행동요령 37
•비상대비물자 준비요령 40
•화생방 대비물자 사용요령 42
•부상자 응급조치 요령 44
01
02
03
04
05
['source', 'page']


In [11]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

# 청크 확인 디버그 코드
for idx, chunk in enumerate(splits):
    print(f"Chunk {idx + 1}:")
    print("-" * 20)
    print(chunk.page_content)
    print("\n" + "=" * 40 + "\n")


Chunk 1:
--------------------
비상시 국민행동요령
CONTENTS
온 가족이 함께 
안전하게
비상사태시 행동요령
화생방 피해대비 
행동요령
인명·시설 피해시 
행동요령
비상대비물자 
준비 및 사용요령
•만화(웹툰)로 보는  02
    비상시 국민행동요령
•비상시 행동요령 08
•민방공 경보 발령시 행동요령 11
•일상생활 비상대비 3가지 14
•화학무기 피해대비 행동요령 20
•생물학무기 피해대비 행동요령 21 
•핵·방사능 피폭대비 행동요령 23
•핵·방사능 피폭대비 생존상식 25
•대형건물 붕괴·화재시 행동요령 32
•전기·물·가스 공급 중단시 행동요령 34
•지하철 피해시 행동요령 35
•인명·시설 피해복구 행동요령 37
•비상대비물자 준비요령 40
•화생방 대비물자 사용요령 42
•부상자 응급조치 요령 44
01
02
03
04
05


Chunk 2:
--------------------
비상시 국민행동요령 알아야 안전하다2
만화로 보는 
비상시 국민
행동요령
온 가족이 함께 보고,
           쉽고 재미있게 배워요!
가정에서, 학교에서, 집 밖에서 언제 어디서나 비상대비!
남녀노소 누구나 어렵지 않게 비상시 행동요령을 배워보아요.
* 비상사태 정의 : 전시, 사변이나 이에 준하는 비상 시(비상대비에 관한 법률 제2조)
엄마가
불길한 꿈을 
꾸었대
비상사태로 다치고,
우리 가족을 잃어버리는
꿈을 꾸었다더구나.
아빠, 엄마
뭐 하시는 거에요?
1 경계 경보가 울리면
이제 끝났다!
평상시 알고 대비하는게
가장 중요해.
당황하지 말고 침착하게 평소 준비해 놓았던 
                       생활필수품을 확인해야 해요.
일상생활 중 잘 대비해 놓고
비상상황에선
침착하게 대응해야 해.
엄마가 공부한
비상시 행동요령들, 
알려줄게!


Chunk 3:
--------------------
3
만화로 보는 비상시 
국민행동요령
화생방 피해대비
행동요령
인명〮시설 피해시
행동요령
비상대비물자
준비 및 사용요령
비상사태시

In [16]:
# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


# 로컬 파일 저장소 설정
store = LocalFileStore("F:\STUDY\sparta\999\박성규\emb")

# 캐시를 지원하는 임베딩 생성 - 임베딩시 계속 api 호출을 방지하기 위해 로컬에 임베팅 파일을 저장하는 형식
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [17]:
vectorstore = FAISS.from_documents(documents=splits, embedding=cached_embedder)

In [18]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 가져올 청크 수를 3으로 늘림

In [20]:
# 프롬프트 템플릿 정의
# contextual_prompt = ChatPromptTemplate.from_messages([
#     ("system", "Answer the question using only the following context."),
#     ("user", "Context: {context}\\n\\nQuestion: {question}")
# ])

In [26]:
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question by combining the provided context and your general knowledge."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])


In [27]:
# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

In [ ]:
while True: 
	print("========================")
	query = input("질문을 입력하세요: ")
	response = rag_chain_debug.invoke(query)
	print("Final Response:")
	print(response.content)

질문을 입력하세요:  비상사태 대처 매뉴얼을 자세히 알려줘


Debug Output: 비상사태 대처 매뉴얼을 자세히 알려줘
Debug Output: {'context': [Document(metadata={'source': '비상시국민행동요령.PDF', 'page': 37}, page_content='비상시 국민행동요령 알아야 안전하다38\n비상사태시 정부에서 국민 구호를\n위한 필요품을 배급하므로 안심하자!\n비상식량과 물은 한 사람이\n30일 정도 버틸 만큼 만!\n05\n비상대비물자 \n준비 및 \n사용요령\n최악의 상황,\n고립된\n상태라도\n버텨야한다\n비상대비물자 준비\n비상대비물자 준비요령\n비상대비물자 40\n비상대비물자 준비요령 41\n가족 비상연락카드 41\n화생방 대비물자 사용요령\n화생방 일반 방독면 착용요령 42\n화생방 대체장비 물자 활용방법 43\n부상자 응급조치 요령\n눈 제염 44\n귀 제염 44\n비강 및 구강 제염 44\n인공호흡 45\n심폐소생술 45\n그렇다고\n사재기는 금물'), Document(metadata={'source': '비상시국민행동요령.PDF', 'page': 35}, page_content='비상시 국민행동요령 알아야 안전하다36\n지하철 객차 내부 \n피해 발생시\n지하철 승강장 \n피해 발생시\n역과 역 사이 또는 \n승강장 화재시\n객차 내부 피해 발생시\n비상 인터폰으로 사고\n내용을 기관사에게 알림\n화재발생시 소화기로\n초기 진화\n출입문 쪽 비상 손잡이를\n수동으로 열고 탈출\n승강장 피해 발생시\n비상전화로 역무실 또는\n사령실에 알림\n비상조명등을 이용하여\n시야를 확보\n비상유도등을 따라\n지상으로 대피\n지하터널 이용 대피\n역과 역 사이 또는\n승강장 화재 발생\n승강장에 있는 비상 사다리를\n이용해 터널로 내려간 후\n열차 진행방향의 선로를 따라\n다음 역으로 이동 대피\n1\n1\n1\n2\n2\n2\n3\n3\n3\n시청 서울역'), Document(metadata={'source': '비상시국민행동요령.PDF', 'page': 0}, p